# Model Deployment
## Introduction

This notebook is using a simple dataset for epitope prediction used in vaccine development from the Kaggle COVID-19/SARS B-cell Epitope Prediction data which cloned on a Github repository for the sake of this project. This notebook will go through the following steps:
1. Processes the model
2. Uploads the model to Watson Machine Learning space
3. Scores a model with test data


## Setup

In [2]:
## Environment libraries
import os, types
import ibm_boto3
from botocore.client import Config
import warnings

## Data libraries and preprocessing
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Machine learning classifier and tools
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV

In [3]:
# Handle warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module", "always" or "once"

## Model processing

In [10]:
# Import Training Set
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_d5592f3b715a4ab696c8411b475cdae6 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_d5592f3b715a4ab696c8411b475cdae6 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_d5592f3b715a4ab696c8411b475cdae6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='qDV6Rk1jMFRAdD1sFE2uZh2tpbGIpHsvc66ObIULfDQ8',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_d5592f3b715a4ab696c8411b475cdae6)

body = client_d5592f3b715a4ab696c8411b475cdae6.get_object(Bucket='covid19epitopeprediction-donotdelete-pr-vjiedfg7ztqsrx',Key='train_data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
print ("Training Set imported ========== 100%")

Training Set imported ========== 100%


In [11]:
# @hidden_cell

# Modeling training data df
# Create variable and target training and testing arrays
X = df.drop(['target'], axis = 1).to_numpy()
y = df['target'].to_numpy()

# Instantiate the machine learning classifiers
mlp = MLPClassifier(hidden_layer_sizes=(50, 100, 50), max_iter=1000, alpha=1e-4, solver='lbfgs', random_state=42, learning_rate_init=.1)

# Splits dataset into training and test set
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)

# Train model
mlp.fit(X_train,y_train)

# Print
print ("Epitope MLP Classifier Model trained ========== 100%")
print("Training set score: %f" % mlp.score(X_train, y_train))

Epitope MLP Classifier Model trained ========== 100%
Training set score: 0.840809


## WML Connection

In [12]:
# @hidden_cell
# Auth information
api_key = 'icFCfbs1_GPNozuI-8Np9ikdsx63MCPzsrfRW1i2ItoR'
location = 'us-south'
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

In [13]:
# create client to access our WML service
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)
print(client.version)

1.0.66


## Space Definition

In [14]:
space_id = '40d50978-e695-4927-a90f-165955ee38f9'

In [15]:
client.spaces.list(limit=10)

------------------------------------  ------------------------  ------------------------
ID                                    NAME                      CREATED
40d50978-e695-4927-a90f-165955ee38f9  COVID Epitope Prediction  2021-04-10T10:00:12.995Z
------------------------------------  ------------------------  ------------------------


In [16]:
client.set.default_space(space_id)

'SUCCESS'

## Upload Model

In [17]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")
metadata = {
            client.repository.ModelMetaNames.NAME: 'COVID 19 Epitope Prediction Model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model = mlp,
    meta_props = metadata)

In [18]:
# Get model details
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "e4429883-c883-42b6-87a8-f419d64088cd",
      "name": "default_py3.7"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "created_at": "2021-04-14T11:19:50.095Z",
    "id": "03ff8bae-0bc0-41e5-bf99-97b5b1de3ecd",
    "modified_at": "2021-04-14T11:19:52.381Z",
    "name": "COVID 19 Epitope Prediction Model",
    "owner": "IBMid-5500032BVE",
    "space_id": "40d50978-e695-4927-a90f-165955ee38f9"
  },
  "system": {
    "warnings": []
  }
}


In [19]:
client.repository.list_models()

------------------------------------  ---------------------------------  ------------------------  -----------------
ID                                    NAME                               CREATED                   TYPE
03ff8bae-0bc0-41e5-bf99-97b5b1de3ecd  COVID 19 Epitope Prediction Model  2021-04-14T11:19:50.002Z  scikit-learn_0.23
ab681a85-f540-40c2-8656-a6265e8d131f  COVID 19 Epitope Prediction Model  2021-04-13T10:40:43.002Z  scikit-learn_0.23
bf0fe8de-9ae9-47aa-8da0-c42422ed04f0  COVID 19 Epitope Prediction Model  2021-04-12T13:21:46.002Z  scikit-learn_0.23
------------------------------------  ---------------------------------  ------------------------  -----------------


In [20]:
# Delete old deployments
##client.repository.delete('GUID of stored model')

## Online Deployment

In [21]:
# Create online deployment
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of COVID 19 Epitope Prediction model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: '03ff8bae-0bc0-41e5-bf99-97b5b1de3ecd' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='897ed470-cb7b-41f8-92ff-d234f0c80191'
------------------------------------------------------------------------------------------------




In [27]:
# Get deployment UID and show details on the deployment
deployment_uid = client.deployments.get_uid(created_deployment)
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': '03ff8bae-0bc0-41e5-bf99-97b5b1de3ecd'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'S', 'num_nodes': 1},
  'name': 'Deployment of COVID 19 Epitope Prediction model',
  'online': {},
  'space_id': '40d50978-e695-4927-a90f-165955ee38f9',
  'status': {'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/897ed470-cb7b-41f8-92ff-d234f0c80191/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-04-14T11:20:23.333Z',
  'id': '897ed470-cb7b-41f8-92ff-d234f0c80191',
  'modified_at': '2021-04-14T11:20:23.333Z',
  'name': 'Deployment of COVID 19 Epitope Prediction model',
  'owner': 'IBMid-5500032BVE',
  'space_id': '40d50978-e695-4927-a90f-165955ee38f9'}}

In [28]:
# list all deployments
client.deployments.list()

------------------------------------  -----------------------------------------------  -----  ------------------------
GUID                                  NAME                                             STATE  CREATED
897ed470-cb7b-41f8-92ff-d234f0c80191  Deployment of COVID 19 Epitope Prediction model  ready  2021-04-14T11:20:23.333Z
1dfcceb8-9a30-4637-a342-3e579f796bdf  C19-Epitope Predictor                            ready  2021-04-13T10:48:01.317Z
------------------------------------  -----------------------------------------------  -----  ------------------------


In [29]:
# delete old deployments
##client.deployments.delete('GUID of deployed model')

## Scoring

In [30]:
# get scoring end point
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/897ed470-cb7b-41f8-92ff-d234f0c80191/predictions


In [31]:
# use our WML client to score our model
# add some test data
scoring_payload = {"input_data": [
    {'fields': ['peptide_length','chou_fasman','emini','kolaskar_tongaonkar','parker','isoelectric_point','aromaticity','hydrophobicity','stability','kmeans_feature'], 
     'values': [[ '0.0','0.476285','0.017313','0.431655','0.619814','0.248550','0.566651','0.564298','0.264627','0.000000' ]]
    }]}

In [32]:
# score the model
predictions = client.deployments.score(deployment_uid, scoring_payload)
print('prediction',json.dumps(predictions, indent=2))

prediction {
  "predictions": [
    {
      "fields": [
        "prediction",
        "probability"
      ],
      "values": [
        [
          1,
          [
            0.20843404578178204,
            0.791565954218218
          ]
        ]
      ]
    }
  ]
}
